In [ ]:
using CSV
using DataFrames
using Glob
using DBFTables
using TableOperations
using Geodesy
using Tables
d = CSV.read("metro_grid.csv", DataFrame)

In [ ]:
files = glob("../webapp/modules/geotools/geotools/data/new_parcels/*.dbf")

In [ ]:
DBFData = DBFTables.Table(files[1]) |> Tables.rowtable

In [ ]:
DBFData[[1,10]]

In [ ]:
lat, lng = [45.147762, -93.134160]

In [ ]:
getDistance(lng, lat, lng, lat)

In [ ]:
d = []
append!(d, 3)

In [ ]:
function getDistance(locLon, locLat, homeLon, homeLat)
    loc = LLA(locLat, locLon, 0)
    home = LLA(homeLat, homeLon, 0)
    return euclidean_distance(loc, home)
end

function addressSearch(homeLon, homeLat, distance)
    files = glob("../webapp/modules/geotools/geotools/data/new_parcels/*.dbf")
    dfs = []
    for file in files
        DBFData = DBFTables.Table(file)
        distances = DBFData |> TableOperations.map(x-> (Dis=getDistance(Tables.getcolumn(x, :LONGITUDE), Tables.getcolumn(x, :LATITUDE), lng, lat),)) |> Tables.columntable;
        rowTable = DBFData |> Tables.rowtable
        filtered = findall(x -> x <= distance, distances[1])
        df = DataFrame(rowTable[filtered])
        df[!, :Dis] = distances[1][filtered]
#         filter!(row => row[:Dis] <= distance, df)
        push!(dfs, df)
    end
    return vcat(dfs)
end

In [ ]:
fullDF = addressSearch(lng, lat, 5000)